In [1]:
from __future__ import print_function, division

In [2]:
import os
import sys
import datetime
import time

import numpy as np
import pandas as pd

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
DATA_DIR = '/Users/appry001/Data/ml-20m/pro/'

In [5]:
unique_uid = list()
with open(os.path.join(DATA_DIR, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
    
unique_sid = list()
with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

In [7]:
n_items = len(unique_sid)
n_users = len(unique_uid)

print(n_users, n_items)

111148 11711


In [10]:
def load_data(csv_file, shape=(n_users, n_items)):
    tp = pd.read_csv(csv_file)
    timestamps, rows, cols = np.array(tp['timestamp']), np.array(tp['uid']), np.array(tp['sid'])
    #seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int'), timestamps[:, None]), axis=1)
    data = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.int16, shape=shape)
    return data #, seq

In [11]:
train_data = load_data(os.path.join(DATA_DIR, 'train.csv'))

In [12]:
vad_data = load_data(os.path.join(DATA_DIR, 'validation.csv'))

In [30]:
test_data = load_data(os.path.join(DATA_DIR, 'test.csv'))

In [18]:
def linear_surplus_confidence_matrix(B, alpha):
    S = B.copy()
    S.data = alpha * S.data
    return S

def log_surplus_confidence_matrix(B, alpha, epsilon):
    S = B.copy()
    S.data = alpha * np.log(1 + S.data / epsilon)
    return S

def recall_k(p_pred, p_true, k=50):
    # p_hat -> p_pred
    # p_vad -> p_true
    m, n = p_pred.shape
    recall = 0
    N = m
    for u in range(m):
        p_true_u = p_true[u].T.toarray().reshape(n)
        nnz = np.sum(p_true_u)
        if nnz < 1:
            N -=1
            continue
        p_pred_u = p_pred[u]
        topk = np.argsort(p_pred_u)[-k:]
        tmp_val = np.sum(p_true_u[topk]) / min(nnz, k)
        if tmp_val > 1:
            return p_true_u, topk, nnz, k
        recall += tmp_val
    return recall / N

In [24]:
def fixed_ALS_update(counts, fix_vectors, user_update=True, n_factors=10, lambda_=0.01):
    '''
    Args:
        counts: sparse csr matrix of user/item consumption counts e_ui: user u consumed item i e_ui times
        fix_vectors: np.ndarray of either item/user features; n_items/n_users by n_factors
        user_update: bool to determine if this will update user or item factor vectors
        n_factors: the number of latent variables to describe users and items
        lambda_: the regularization of the user and item factor vectors.
    Returns:
        solve_vecs: np.ndarray of either user/item features
        
    Notes:
        Implicit matrix factorization predicts preferences by using the confidence and preference of each
        user consuming items where counts[u] denotes the consumption of all items, viz.
        pref[u] = 1 if counts[u] > 0 else 0.
        conf[u] = 1 + alpha * counts[u].
        Unobserved values(zeroes in counts[u]) get confidence 1 and observed values get confidense
        proportional to counts[u].
        
        Using alternating least squares (ALS), we solve the linear system,
        (YTCuY + lambda_I) xu = YTCupu,
        where YT is Y.transpose and Cu is a diagonal matrix with conf[u] along the diagonal.
        Because conf[u] is dense, the diagonal of Cu will also be dense.
        To speed up the solution to the linear system, we do not use Cu or YTCuY and compose instead
        {YTY + YT(Cu-I)Y + lambda_I} xu = YT(Cu-I+I)Y
        The precomputation of YTY computation of Cu-I, `Cu_minus_I`, for each user makes our solution
        scale with only the observed counts. Concretely, while the initial YTCuY computation would 
        scale with n_items for each user, computation of YT(Cu-I)Y scales with n_i,
        the number of items user u consumed, n_i << n_items.
        
    '''
    m, n = counts.shape
    num_solve = m if user_update else n
    
    fixed = sparse.csr_matrix(fix_vectors)
    num_fixed = fixed.shape[0]

    solve_vecs = np.zeros((num_solve, n_factors))
    
    # precompute user independent objects
    YTY = np.dot(fixed.T, fixed)
    eye = sparse.eye(num_fixed)
    lambda_eye = lambda_ * sparse.eye(n_factors)

    for u in xrange(num_solve):
        if user_update:
            c_u = counts[u].toarray()
        else:
            c_u = counts[:, u].T.toarray()
        pu = c_u.copy()
        pu[np.where(pu != 0)] = 1.0
        
        Cu_minus_I = sparse.diags(c_u, [0])        
        YTCuIY = fixed.T.dot(Cu_minus_I).dot(fixed)
        YTCupu = fixed.T.dot(Cu_minus_I + eye).dot(sparse.csr_matrix(pu).T)
        xu = spsolve(YTY + YTCuIY + lambda_eye, YTCupu)
        solve_vecs[u] = xu
        
    return solve_vecs

In [25]:
n_factors = 10
num_iterations = 3
lambda_ = 1e-5

alpha = 10
counts = linear_surplus_confidence_matrix(train_data, alpha)

# random init of user/item factors
user_vectors = np.random.normal(size=(n_users, n_factors))
item_vectors = np.random.normal(size=(n_items, n_factors))

print(user_vectors.shape, item_vectors.shape)

(111148, 10) (11711, 10)


In [31]:
p_train = train_data.copy()
p_train.data = np.ones_like(train_data.data)
p_train

p_vad = vad_data.copy()
p_vad.data = np.ones_like(vad_data.data)
p_vad

p_test = test_data.copy()
p_test.data = np.ones_like(test_data.data)
p_test

<111148x11711 sparse matrix of type '<type 'numpy.int16'>'
	with 207161 stored elements in Compressed Sparse Row format>

In [29]:
for _ in xrange(num_iterations):
    t = time.time()
    user_vectors = fixed_ALS_update(
        counts, item_vectors, True, n_factors=n_factors, lambda_=lambda_)
    print('\tUser update...{0:5.5}s '.format(time.time() - t))
    t = time.time()
    item_vectors = fixed_ALS_update(
        counts, user_vectors, False, n_factors=n_factors, lambda_=lambda_)
    print('\tItem update...{0:5.5}s '.format(time.time() - t))
    
    # calc metric
    p_pred = user_vectors.dot(item_vectors.T)
    score = recall_k(p_pred, p_vad, k=50)
    print('Recall@50= {0:.3}'.format(score))

	User update...455.45s 
	Item update...494.35s 
Recall@50= 0.348
	User update...453.1s 
	Item update...495.69s 
Recall@50= 0.365
	User update...451.1s 
	Item update...500.1s 
Recall@50= 0.368


In [28]:
# calc metric
p_pred = user_vectors.dot(item_vectors.T)
score = recall_k(p_pred, p_vad, k=50)
print('Recall@50= {0:.3}'.format(score))

Recall@50= 0.185


In [32]:
# calc metric
p_pred = user_vectors.dot(item_vectors.T)
score = recall_k(p_pred, p_test, k=50)
print('Recall@50= {0:.3}'.format(score))

Recall@50= 0.0956
